In [ ]:
import time
import requests
import aiohttp
import asyncio
from concurrent.futures import ThreadPoolExecutor
import multiprocessing as mp

# ------------------------------
# Función de descarga (común a todos)
# ------------------------------
def download_image(url, filename):
    resp = requests.get(url, timeout=10)  # máximo 10 seg por request
    with open(filename, "wb") as f:
        f.write(resp.content)
    return f"{filename} descargada"
# Lista de imágenes (ejemplo)
IMAGES = [
    "https://atlnacional.com.co/wp-content/webp-express/webp-images/uploads/2022/02/Mascota.jpg.webp",    

]


In [8]:
# ------------------------------
# Multiprocessing
# ------------------------------
def download_with_multiprocessing():
    with mp.get_context("spawn").Pool() as pool:
        results = pool.starmap(
            download_image,
            zip(IMAGES, [f"mp_{i}.jpg" for i in range(len(IMAGES))])
        )
    return results


In [9]:
# ------------------------------
# Threading
# ------------------------------
def download_with_threads():
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(
            download_image,
            IMAGES,
            [f"th_{i}.jpg" for i in range(len(IMAGES))]
        ))
    return results

In [10]:
# ------------------------------
# Asyncio
# ------------------------------
async def async_download_image(session, url, filename):
    async with session.get(url) as resp:
        content = await resp.read()
        with open(filename, "wb") as f:
            f.write(content)
    return f"{filename} descargada"

async def download_with_asyncio():
    async with aiohttp.ClientSession() as session:
        tasks = [
            async_download_image(session, url, f"async_{i}.jpg")
            for i, url in enumerate(IMAGES)
        ]
        return await asyncio.gather(*tasks)

In [11]:

def medir_tiempo(func, is_async=False):
    start = time.perf_counter()
    
    if is_async:
        result = asyncio.run(func())
    else:
        result = func()
    
    end = time.perf_counter()
    print(f"\n{func.__name__} terminó en {end - start:.2f} segundos")
    print("Resultados:", result)
    return result

In [ ]:
# ------------------------------
print("Comparación de concurrencia:\n")

medir_tiempo(download_with_multiprocessing)
medir_tiempo(download_with_threads)
medir_tiempo(download_with_asyncio, is_async=True)

Comparación de concurrencia:

